<a href="https://colab.research.google.com/github/JJay1010/K-software_-/blob/main/SeojeanKang/drug/drug_230112_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**일시**: 2023. 01. 12 (01.10~01.12 진행)

**사용 데이터셋**: 경구약제 조합 5,000 조합 training - (TS_4_조합.zip, TL_4_조합.zip)

(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=576)


**데이터셋 구축 목표**: 국내 유동되고 있는 경구약제 5,000종을 대상으로 약품식별 인공지능 개발 및 학습

**환경**: 데이터 업로드하여 colab으로 작업

---
**개인 목표**: 이미지 데이터 전처리 연습


**실행 내용**: 약제 조합 이미지 데이터 전처리 및 시각화 

> img_to_json 으로 이미지 크롭하여 저장
>
> json 내용을 DataFrame으로 만들고, csv 파일로 저장하기
>
> 크롭한 이미지 tensor로 바꿔보기...

**발생한 문제**: 

1) 이미지 파일 중 ~index.png는 크롭이 불가능하여 오류

2) csv 저장할 때 한글이 깨지는 오류

3) annotation에 bbox 중 일부 , 가 .로 입력된 오류

**해결책**: 

1) re 수정하여 ~index.png를 제외한 사진 경로 리스트를 만듦

2) utf-8이 아닌 utf-8-sig로 인코딩하여 해결

3) bbox에 replace와 split으로 오류 수정 --> 새로운 오류 발견 "ValueError: invalid literal for int() with base 10: ''" ... 우선 100개만 끊어서 진행


##### import & unzip

In [24]:
import os
import json
import matplotlib.pyplot as plt
import PIL as pil
from PIL import Image
import re
import pandas as pd
from google.colab import files

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data

pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount("/content/mydrive")

# 라벨링 데이터 압축 해제
!mkdir druglabel
!cd '/content/druglabel' # 압축을 풀 폴더 경로
!unzip -qq '/content/mydrive/MyDrive/jan_prj/TL_4_조합.zip' -d '/content/druglabel' # 압축파일 경로

# 사진 데이터 압축 해제
!mkdir drugphoto
!cd '/content/drugphoto' # 압축을 풀 폴더 경로
!unzip -qq '/content/mydrive/MyDrive/jan_prj/TS_4_조합.zip' -d '/content/drugphoto' # 압축파일 경로

Mounted at /content/mydrive


##### label, photo 데이터 압축 풀기 및 경로 리스트화

In [4]:
# json 레이블 경로 리스트로 만들기

filepath = 'druglabel/'

labeldir1 = [os.path.join(filepath, i) for i in os.listdir(filepath) if os.path.isdir(os.path.join(filepath, i))]

labeldir2=[]
for i in labeldir1:
  fdir = os.listdir(i)
  for j in fdir:
    if os.path.isdir(os.path.join(i, j)):
      labeldir2.append(i + '/' + j)

labeldir3 = []
for i in labeldir2:
  fdir = os.listdir(i)
  for j in fdir:
    labeldir3.append(i + '/' + j)

len(labeldir3)

5467

In [5]:
# 사진 데이터 경로 리스트로 만들기
photopath = 'drugphoto/'

photodir1 = [os.path.join(photopath, i) for i in os.listdir(photopath) if os.path.isdir(os.path.join(photopath, i))]

photodir2=[]
for i in photodir1:
  fdir = os.listdir(i)
  for j in fdir:
    photodir2.append(i + '/' + j)

len(photodir2)

2000

##### img 파일로 json 불러 크롭

In [ ]:
# 발견한 문제: index 그림은 크롭할 수 없음. 
# r'(K-[\w-][^index]+)\.png'로 수정하여 적용
# photodir2에서 index 사진 삭제. 

pattern2 = r'(drugphoto\/K-[\w-][^index]+\.png)'

photodir3 = []
for i in photodir2:
  dir = re.findall(pattern2, i)
  if dir:
    photodir3.append(dir)

# 1차원으로 변경
photodir3 = [i[0] for i in photodir3]

'drugphoto/K-004543-010221-016548-021771/K-004543-010221-016548-021771_0_2_0_2_90_000_200.png'

In [ ]:
# # bbox 일부 , 가 .로 입력된 오류
# . 를 ,로 replace --> split으로 다시 리스트로 만듦 --> int 적용
# 그랬더니 ValueError: invalid literal for int() with base 10: ''

for k in photodir3[:10]:
  img = Image.open(k)

  pattern = r'(K-[\w-]+)\.png'
  img_title = re.findall(pattern, k)
  if img_title:
    labels = [i for i in labeldir3 if img_title[0] in i]

  for i in range(len(labels)):
    label = open(labels[i])
    label = json.load(label)
    lbl_ant = label.get('annotations')
    bbox = lbl_ant[0]['bbox']
    bbox = str(bbox).replace('.',',')[1:-1]
    bbox = bbox.split(',')
    bbox = [int(i) for i in bbox]
    print(bbox, end=' / ')

[631, 169, 218, 213] / [120, 234, 209, 201] / [113, 781, 234, 226] / [627, 816, 191, 186] / [125, 875, 232, 228] / [664, 831, 216, 213] / [643, 293, 235, 225] / [185, 306, 188, 181] / [602, 165, 215, 214] / [105, 223, 202, 198] / [81, 737, 237, 234] / [601, 782, 192, 190] / [108, 195, 210, 201] / [649, 793, 213, 204] / [601, 98, 282, 481] / [97, 843, 331, 157] / [676, 869, 220, 211] / [141, 308, 206, 198] / [98, 664, 292, 494] / [566, 304, 328, 159] / [92, 187, 205, 199] / [624, 761, 213, 202] / [574, 81, 276, 472] / [65, 804, 329, 158] / [369, 247, 183, 181] / [593, 816, 201, 274] / [167, 609, 260, 662] / [424, 856, 189, 181] / [89, 240, 304, 264] / [545, 56, 261, 608] / [329, 189, 183, 180] / [550, 720, 309, 282] / [122, 517, 265, 650] / [630, 827, 221, 217] / [193, 711, 219, 358] / [589, 159, 240, 229] / [167, 202, 188, 185] / 

In [ ]:
def img_to_json(imgfile):# 크롭한 이미지 리스트로 반환
  img = Image.open(imgfile)

  # img 경로에서 img_title 제목 추출
  pattern = r'(K-[\w-]+)\.png'
  img_title = re.findall(pattern, imgfile)
  if img_title:
    labels = [i for i in labeldir3 if img_title[0] in i]

    cropped = []

      # image crop
    for i in range(len(labels)):
      label = open(labels[i])
      label = json.load(label)
      lbl_ant = label.get('annotations')
      bbox = lbl_ant[0]['bbox']
      bbox = str(bbox).replace('.',',')[1:-1]
      bbox = bbox.split(',')
      bbox = [int(i) for i in bbox]
      
      x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]

      cropped_img = img.crop((x, y, x+w, y+h))
      cropped.append(cropped_img)
    
    return cropped

In [ ]:
# json labels만 추출
def img_to_labels(imgfile):
  img = Image.open(imgfile)

  pattern = r'(K-[\w-]+)\.png'
  img_title = re.findall(pattern, imgfile)
  if img_title:
    labels = [i for i in labeldir3 if img_title[0] in i]
  return labels

##### DataFrame을 csv 파일로 저장하기

In [3]:
# imgfile 데이터를 받아 해당하는 json을 DataFrame화하는 함수
def img_to_df(imgfile):
  img = Image.open(imgfile)

  pattern = r'(K-[\w-]+)\.png'
  img_title = re.findall(pattern, imgfile)
  
  labels = [i for i in labeldir3 if img_title[0] in i]

  img_info = []

  for i in range(len(labels)):
    label = open(labels[i])
    label = json.load(label)

    # json dataframe
    # json에서 'image' 골라내 리스트화
    lbl_image = label.get('images')
    img_info.append(lbl_image[0])
    
  df = pd.json_normalize(img_info[0])
  for info in img_info[1:]:
    temp = pd.json_normalize(info)
    df = df.append(temp)
    df = df.reset_index(drop=True)

  return df       # json images 데이터프레임으로 반환

In [6]:
# 0번 인덱스 데이터로 칼럼 만들 겸 데이터프레임 만들기
json_img = open(labeldir3[0])
json_img = json.load(json_img)
df_img = json_img.get('images')
df_img = pd.json_normalize(df_img)
df_img

,file_name,width,height,imgfile,drug_N,drug_S,back_color,drug_dir,light_color,camera_la,camera_lo,size,dl_idx,dl_mapping_code,dl_name,dl_name_en,img_key,dl_material,dl_material_en,dl_custom_shape,dl_company,dl_company_en,di_company_mf,di_company_mf_en,item_seq,di_item_permit_date,di_class_no,di_etc_otc_code,di_edi_code,chart,drug_shape,thick,leng_long,leng_short,print_front,print_back,color_class1,color_class2,line_front,line_back,img_regist_ts,form_code_name,mark_code_front_anal,mark_code_back_anal,mark_code_front_img,mark_code_back_img,mark_code_front,mark_code_back,change_date,id
0,K-005000-020238-031863_0_2_0_2_90_000_200.png,976,1280,K-005000-020238-031863_0_2_0_2_90_000_200.png,K-031863,정상알약,연회색 배경,앞면,주백색,90,0,200,31862,K-031863,아질렉트정(라사길린메실산염),Azilect Tab.,http://connectdi.com/design/img/drug/147426780...,라사길린메실산염,Rasagiline Mesylate,"정제, 저작정",한국룬드벡(주),Lundbeck Korea,Teva Pharmaceutical Industries Ltd.,,201309347,20130913,[01190]기타의 중추신경용약,전문의약품,668000100,"한쪽 면의 위쪽에 “GIL"", 아래쪽에 ”1“이 새겨진 흰색 또는 거의 흰색의 모서...",원형,3,8,8,GIL 1,,하양,,,,20131224,나정,,,,,,,20210106,1


In [7]:
# DataFrame 작성
for i in range(1, len(labeldir3)):
  js = open(labeldir3[i])
  try:
    js = json.load(js)
    jsdf = js.get('images')
    jsdf = pd.json_normalize(jsdf)
    df_img = df_img.append(jsdf)
  except UnicodeDecodeError:
    continue

df_img = df_img.reset_index(drop=True)

In [27]:
# DataFrame을 csv로 저장 & 다운로드
# encoding='utf-8' 로 해도 한글이 깨짐 --> encoding='utf-8-sig' 로 해결
df_img.to_csv('drug_data.csv', encoding='utf-8-sig')
files.download('drug_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### 텐서화

In [ ]:
# ValueError: invalid literal for int() with base 10: ''
# 이미지 전부 크롭하려고 했더니 어딘가에서 오류... 100개로 끊기

individual = [] # [[cropped]]
for i in photodir3[:100]:
    individual.append(img_to_json(i))

len(individual)

100

In [ ]:
idv_crop =[i[j] for i in individual for j in range(len(i))]

len(idv_crop)
# 358개의 크롭된 이미지

358

In [ ]:
lbl = []
for i in photodir3[:100]:
    lbl.append(img_to_labels(i))

lbls = [i[j] for i in lbl for j in range(len(i))]
len(lbls) # 이미지와 레이블 개수 일치

358

In [ ]:
transform_drug = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((64, 64))])

# tf = transform_drug(idv_crop[0])
# tf.size() >> torch.Size([3, 64, 64])

img_t = [transform_drug(i) for i in idv_crop]
len(img_t)

358

In [ ]:
# train_loader = data.DataLoader(
#     dataset = (img_t, lbls),
#     batch_size = 20, shuffle = True
# )

# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# images.shape, labels.shape